

# 1. Install Java and NLU


In [ ]:
import os
import pandas as pd
import numpy as np

! apt-get update -qq > /dev/null   
# Install jav
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install nlu pyspark==3.3 > /dev/null


In [ ]:
import nlu
pipe = nlu.load('xlnet ')

# 2. Load data

In [ ]:
from google.colab import files
df=pd.read_csv("./Abstracts-kwlg-Parsed.csv", sep=',')
#df=pd.read_csv("./Methods-Parsed.csv", sep=',')
df=df.rename(columns={'Content_Parsed': 'text'})

In [ ]:
import re
import unicodedata
for i in range (len(df['text'])):
  df['text'][i] = unicodedata.normalize('NFKD', df['text'][i]).encode('ascii', 'ignore').decode("utf-8")
  df['text'][i] = re.sub(r'[^\w]', ' ', df['text'][i])


#  3. Load Model and Embed sample string

In [ ]:
import numpy as np
embeddings = []
all_embeddings = []
sentence_embedding = np.empty(768, dtype=object)
c=0

for txt in df['text']:
  print("*****************item ",c)
  embedding = pipe.predict([txt])
  #embedding = embedding.reset_index(drop=True)
  flatkeys=[element for sublist in embedding['token'].tolist() for element in sublist]
  flatvalues=[element for sublist in embedding['word_embedding_xlnet'].tolist() for element in sublist]
  embedding = dict(zip(flatkeys, flatvalues))
  embeddings.append(embedding)
  c=c+1


for l in range(len(embeddings)):
  for v in embeddings[l].values():
    sentence_embedding=np.vstack((sentence_embedding, v))
  sentence_embedding = np.delete(sentence_embedding, obj=0, axis=0)
  sentence_embedding = (np.mean(sentence_embedding, axis=0)).tolist()
  all_embeddings.append(sentence_embedding)

all_embeddings = np.array(all_embeddings)
all_embeddings = pd.DataFrame(all_embeddings)

all_embeddings.insert(loc=0, column='text', value=df['text'])

*****************item  0
*****************item  1
*****************item  2
*****************item  3
*****************item  4
*****************item  5
*****************item  6
*****************item  7
*****************item  8
*****************item  9
*****************item  10
*****************item  11
*****************item  12
*****************item  13
*****************item  14
*****************item  15
*****************item  16
*****************item  17
*****************item  18
*****************item  19
*****************item  20
*****************item  21
*****************item  22
*****************item  23
*****************item  24
*****************item  25
*****************item  26
*****************item  27
*****************item  28
*****************item  29
*****************item  30
*****************item  31
*****************item  32
*****************item  33
*****************item  34
*****************item  35
*****************item  36
*****************item  37
*****************item 

In [ ]:
all_embeddings.to_csv('Abstracts-kwlg-XLNET.csv') 
files.download("Abstracts-kwlg-XLNET.csv")